In [1]:
from package import *

import requests
from pprint import pprint
from functools import partial
import pandas
import matplotlib.pyplot as plt
import os
import sys
import datetime
from pony.orm import db_session
from fuzzywuzzy import process
github_folder = os.path.join(os.getenv('USERPROFILE'), 'Documents', 'Github', 'python-youtube-api')
sys.path.append(github_folder)

pprint = partial(pprint, width = 120)

#youtube = YouTubeDatabase(github.youtube_api_key, 'youtube_database')
killian = "UCjdQaSJCYS4o2eG93MvIwqg"
test_video = "836Lv_swaCk"
roosterteeth = 'UCzH3iADRIq1IJlIXjfNgTpA'
vs_playlist = "PLTur7oukosPH7Kvex9-rUiZgVIVGt6C--"
youtube = YouTube(github.youtube_api_key)
youtubedb = YoutubeDatabase(youtube, 'youtube_database')
print("Finished!")

@db_session

def listChannelNames(database):
    for channel in database.Channel.select(lambda s: s):
        print(channel.channelName)
#listChannelNames(youtubedb)

Finished!


C:\Users\Progi\Anaconda3\lib\site-packages\fuzzywuzzy-0.16.0-py3.6.egg\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning


In [38]:

@db_session
def findLongest(database):
    channel = database.Channel.get(channelName = 'LetsPlay')
    cuttoff = datetime.timedelta(seconds = 3000)
    query = channel.video.select(lambda s: s)
    for video in query.order_by(lambda s: s.videoDuration):
        if video.videoDuration < cuttoff:
            continue

        try:
            title, series = video.videoName.split('-')
        except ValueError:
            title = video.videoName
            series = ''

        print("{}\t{:<30}\t{}".format(video.videoDuration, series,title))
findLongest(youtubedb)

0:50:10	 Mario Party 8: DK's Treetop Temple Part 1	Let's Play 
0:50:14	 Cloudberry Kingdom Part 5    	Let's Play 
0:50:52	                              	Full Play - Banjo-Tooie Part 4
0:51:03	 Journey to the (South)West   	Let's Play Minecraft: Ep. 237 
0:51:20	 Wither Hunt                  	Let's Play Minecraft: Ep. 140 
0:52:00	 Hot Seat: Prop Hunt Featuring Max Kruemcke	Let's Play 
0:52:15	 Halloween Spooktacular Part 1	Let's Build in Minecraft 
0:52:18	 Polar Express                	Let's Build in Minecraft 
0:52:35	                              	Let's Play - Golf With Your Friends: AH Live Stream - Part 2
0:53:11	                              	Let's Play Minecraft: Ep. 169 - Re-Find the Tower
0:53:38	                              	Let's Play Minecraft - Episode 296 - Sky Factory Part 35
0:53:55	 Cloudberry Kingdom Part 18   	Let's Play 
0:54:10	 7 Days to Die for PC: Part 3 	Let's Play 
0:55:13	                              	Let's Play - Splinter Cell: Blacklist - Co-op Part 3
0:5

In [50]:
from typing import *
@db_session
def analyzeCharacter(database, person: str, channel_whitelist: Optional[List[str]]=None):
    from fuzzywuzzy import process
    avideos = database.Video.select(lambda s: s.channel.channelName in channel_whitelist)
    all_tags = avideos.tags
    all_tags = [i.string for i in all_tags]
    similar_tags = process.extract(person, all_tags, limit = 10)
    pprint(similar_tags)
    videos = list()
    for element in similar_tags:
        tag, _ = element 
        videos += [i for i in database.Tag.get(string = tag).videos]
    
    videos = sorted(videos, key = lambda s: (s.channel.channelName, s.videoName))
    for video in videos:
        duration = video.videoDuration
        channel_name = video.channel.channelName
        try:
            title, series = video.videoName.split('-')
        except ValueError:
            title = video.videoName
            series = ''
        print("{}\t{:<30}\t{}".format(duration, channel_name, title))
    
analyzeCharacter(youtubedb, 'andrew blanchard', ['Rooster Teeth', 'Achievement Hunter', 'LetsPlay'])

AttributeError: 'Query' object has no attribute 'tags'

In [2]:
from fuzzywuzzy import process
ALL_TAGS = sorted(set([i.string.lower() for i in youtubedb.Tag.select(lambda s: s)]))
@db_session
def findSimilarTags(database, tags):
    if isinstance(tags, str):
        tags = [tags]
    all_tags = ALL_TAGS
    for tag in tags:
        tag = tag.lower()
        print(tag)
        similar_tags = process.extract(tag, all_tags)
        for j in similar_tags:
            print('\t',j)
findSimilarTags(youtubedb, ['lindsey', 'lindsey jones'])

lindsey
	 ('lindsey', 100)
	 ("'n", 90)
	 ("'n'", 90)
	 (':d', 90)
	 ('ds', 90)
lindsey jones
	 ('lindsey jones', 100)
	 ('lindasy jones', 92)
	 ('lindsay jones', 92)
	 ('ds', 90)
	 ('in', 90)


In [3]:
#'Achievement Hunter'
@db_session
def getVideosByTag(database, tags, channels = None):
    if isinstance(tags, str):
        tags = [tags]
    tags = set(i.lower() for i in tags)
    
    tag_entities = database.Tag.select(lambda s: s.string.lower() in tags)
    
    videos = list()
    for tag in tag_entities:
        videos += list(VideoResource.fromSql(i.toDict()) for i in tag.videos)
    for vid in sorted(videos, key = lambda s: s['videoName']):
        if vid is not None and vid['channelName'] not in channels:
            continue
        print(vid)
        
getVideosByTag(youtubedb, ['lindsey', 'lindsey jones', 'lindsay jones'], ['Achievement Hunter', 'LetsPlay'])

VideoResource('yWmW_kBIp0c', 'Achievement Hunter', '2016 Achievement Hunter Hockey League: Southeast Pimberton Division - Week 4')
VideoResource('CYjBU_Qd0ew', 'Achievement Hunter', 'AH Animated - Geoff Kills Mike')
VideoResource('ATruJkvKOBc', 'Achievement Hunter', 'AH Animated - Michael Marries Gavin')
VideoResource('QIwjxv8m5Ok', 'Achievement Hunter', 'AH Animated - The Animal Lover')
VideoResource('6NVK43Xod7o', 'Achievement Hunter', 'AH Animated - The Garbo Family')
VideoResource('B8VJ4FVDUuk', 'LetsPlay', 'ASMR-DoNotUpload.MOV')
VideoResource('tlEeLP6OfPU', 'Achievement Hunter', 'Achievement Hunter VS The World - Rocket League Rematch')
VideoResource('He4-ypHlzpo', 'Achievement Hunter', 'Axe Box-ing - AHWU for January 8th, 2018 (#403)')
VideoResource('hbnBiiuj3-U', 'Achievement Hunter', 'Baggin' Gavin - AHWU for March 13th, 2017 (#360)')
VideoResource('P_xcCQqEEcc', 'Achievement Hunter', 'Best of Achievement Hunter - 2017')
VideoResource('We8Or3jAhcM', 'Achievement Hunter', 'Best

In [6]:
test = VideoResource.fromSql(youtubedb.Video['JWRPaJJxPtA'].toDict())
print(test['videoName'])
print(test['videoDuration'])
print(test['videoDescription'])
print(sorted(test['videoTags']))

Let's Play - DOOM CoOp: Part 1
PT27M9.0S
A team of space marines known as Team Jumbo Shrimp, made up of Jack and Jeremy, are flown to the moon of Mars, Phobos. There, they have to fight the forces of Hell that have burst through to try and get their signatures. But Jumbo Shrimp aint got time to sign all of those Imp t-shirts. Do you know how many sleeve holes they have for those spikes? It's stupid! Anyways... DOOM with Jack and Jeremy! Fight Demons Alongside Jack and Jeremy! http://bit.ly/23OwhLq

» Join FIRST to watch episodes early: http://bit.ly/2vv4OWI
» Get your Let's Play merch: http://bit.ly/2vJi7yQ
» Subscribe: http://bit.ly/1BuRgl1

About Let's Play:
A revolving cast of Rooster Teeth employees play video games extremely poorly.

More from Let's Play:
» Achievement Hunter: http://bit.ly/AHYTChannel

Let's Play - DOOM CoOp: Part 1
https://www.youtube.com/user/letsplay
['AH', 'Achievement', 'Achievement Hunter', 'Funhaus', 'GO!', 'Gavin Free', 'Gavin Free (Person)', 'Geoff Ramse